In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

Define Paths and Create Directories

In [2]:
data_dir = Path("../data/processed")
models_dir = Path("../models")
results_dir = Path("../results")
results_dir.mkdir(exist_ok=True)
figures_dir = results_dir / "figures"
figures_dir.mkdir(exist_ok=True)

Define Helper Functions to Recreate the Featured Dataframe

In [4]:
def create_aligned_dataframe(nwm_df, usgs_df):
    """Aligns NWM forecasts with USGS observations and calculates error."""
    nwm_df = nwm_df.sort_values(by='model_output_valid_time')
    usgs_df = usgs_df.sort_index()
    aligned_df = pd.merge_asof(
        left=nwm_df, right=usgs_df,
        left_on='model_output_valid_time', right_index=True,
        direction='nearest', tolerance=pd.Timedelta(minutes=30)
    )
    aligned_df.dropna(subset=['USGSFlowValue'], inplace=True)
    aligned_df = aligned_df.rename(columns={
        'streamflow_value': 'NWM_streamflow', 'USGSFlowValue': 'USGS_streamflow'
    })
    return aligned_df

def create_features(df):
    """Creates the target variable and input features."""
    df['error'] = df['NWM_streamflow'] - df['USGS_streamflow']
    valid_time = df['model_output_valid_time']
    df['month'] = valid_time.dt.month
    df['day_of_year'] = valid_time.dt.dayofyear
    df['hour'] = valid_time.dt.hour
    feature_cols = [
        'error', 'NWM_streamflow', 'USGS_streamflow', # Also include USGS for later
        'lead_time', 'month', 'day_of_year', 'hour'
    ]
    df_featured = df[['model_output_valid_time'] + feature_cols].set_index('model_output_valid_time')
    return df_featured


Load Data and Recreate Featured Dataframe for each Station

In [6]:
station_data = {}
for station_name in ['station1', 'station2']:
    # Load the raw component data
    nwm_df = pd.read_parquet(data_dir / f"{station_name}_nwm.parquet")
    usgs_df = pd.read_parquet(data_dir / f"{station_name}_usgs.parquet")
    
    # Ensure datetime types match for merging
    if nwm_df['model_output_valid_time'].dt.tz is None:
        nwm_df['model_output_valid_time'] = nwm_df['model_output_valid_time'].dt.tz_localize('UTC')
    # Recreate the aligned and featured dataframe
    aligned_df = create_aligned_dataframe(nwm_df, usgs_df)
    featured_df = create_features(aligned_df)

    # Load the sequenced test data
    data_for_modeling = np.load(data_dir / f'{station_name}_processed_for_modeling.npz')
    # Load the scaler
    with open(data_dir / f'scaler_{station_name}.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    station_data[station_name] = {
        'X_test': data_for_modeling['X_test'],
        'y_test_scaled': data_for_modeling['y_test'],
        'scaler': scaler,
        'original_featured_df': featured_df # This now correctly holds the data we need
    }

Define the Metrics Calculation Function

In [7]:
def calculate_metrics(observed, forecasted):
    """Calculates CC, RMSE, PBIAS, and NSE."""
    observed = np.array(observed)
    forecasted = np.array(forecasted)
    
    # Coefficient of correlation (CC)
    cc = np.corrcoef(observed, forecasted)[0, 1]
    
    # Root mean square error (RMSE)
    rmse = np.sqrt(np.mean((forecasted - observed) ** 2))
    
    # Percent bias (PBIAS)
    pbias = 100 * np.sum(forecasted - observed) / np.sum(observed)
    
    # Nash-Sutcliffe Efficiency (NSE)
    nse = 1 - (np.sum((forecasted - observed) ** 2) / np.sum((observed - np.mean(observed)) ** 2))
    
    return {'CC': cc, 'RMSE': rmse, 'PBIAS': pbias, 'NSE': nse}

print("Setup complete. All data, models, and scalers are ready.")

Setup complete. All data, models, and scalers are ready.


Loop through each model, make predictions, and calculate the metrics

In [9]:
all_results = {}
LOOKBACK = 24 # Must be the same as in the preprocessing step

for station_name in ['station1', 'station2']:
    print(f"\n{'='*60}\n///// Evaluating models for {station_name} /////\n{'='*60}")
    all_results[station_name] = {}
    
    # Get the data for the current station
    data = station_data[station_name]
    X_test = data['X_test']
    scaler = data['scaler']
    
    # We need the original test dataframe to get unscaled values and lead times
    test_start_date = '2022-10-01 00:00:00'
    original_test_df = data['original_featured_df'].loc[test_start_date:].iloc[LOOKBACK:]

    for model_name in ['lstm', 'gru', 'transformer']:
        print(f"\n--- Evaluating {model_name.upper()} model ---")
        
        # Load the trained model
        model = keras.models.load_model(models_dir / f"{station_name}_{model_name}_model.keras")
        
        # 1. Make predictions (these are the scaled errors)
        predicted_error_scaled = model.predict(X_test)
        
        # 2. Inverse scale the predictions
        # Create a dummy array with the same shape as the input to the scaler
        dummy_array = np.zeros((len(predicted_error_scaled), X_test.shape[2]))
        dummy_array[:, 0] = predicted_error_scaled.ravel() # Put predictions in the first column ('error')
        predicted_error_unscaled = scaler.inverse_transform(dummy_array)[:, 0]
        
        # 3. Create a results DataFrame
        results_df = original_test_df.copy()
        results_df['predicted_error'] = predicted_error_unscaled
        
        # 4. Calculate the corrected forecast
        # Corrected_Forecast = NWM_streamflow - predicted_error
        # Note the minus sign, because our error = NWM - USGS
        results_df['Corrected_Forecast'] = results_df['NWM_streamflow'] - results_df['predicted_error']

        # 5. Calculate metrics for each lead time
        metrics_by_lead_time = []
        for lead_time, group in results_df.groupby('lead_time'):
            if len(group) < 2: continue # Need at least 2 points to calculate correlation
                
            # Baseline metrics (Original NWM vs. Observed)
            baseline_metrics = calculate_metrics(group['USGS_streamflow'], group['NWM_streamflow'])
            baseline_metrics['lead_time'] = lead_time
            baseline_metrics['type'] = 'NWM (Baseline)'
            
            # Corrected forecast metrics (Our Model vs. Observed)
            corrected_metrics = calculate_metrics(group['USGS_streamflow'], group['Corrected_Forecast'])
            corrected_metrics['lead_time'] = lead_time
            corrected_metrics['type'] = f'Corrected ({model_name.upper()})'
            
            metrics_by_lead_time.extend([baseline_metrics, corrected_metrics])
            
        metrics_df = pd.DataFrame(metrics_by_lead_time)
        
        # Store results for plotting
        all_results[station_name][model_name] = {'results_df': results_df, 'metrics_df': metrics_df}
        print(f"Evaluation complete for {model_name.upper()}.")

# Save the final results dictionary
with open(results_dir / 'all_evaluation_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)


///// Evaluating models for station1 /////

--- Evaluating LSTM model ---


ValueError: File not found: filepath=..\models\station1_lstm_model.keras. Please ensure the file is an accessible `.keras` zip file.

Visualization Loop

In [ ]:
for station_name, models in all_results.items():
    for model_name, data in models.items():
        results_df = data['results_df']
        metrics_df = data['metrics_df']
        
        # --- 1. Runoff Box Plot ---
        plt.figure(figsize=(18, 8))
        
        # Prepare data for boxplot
        plot_data = results_df[['lead_time', 'USGS_streamflow', 'NWM_streamflow', 'Corrected_Forecast']]
        plot_data = plot_data.melt(id_vars='lead_time', var_name='Type', value_name='Flow')
        plot_data.rename(columns={'Type': 'Forecast Type'}, inplace=True)

        sns.boxplot(data=plot_data, x='lead_time', y='Flow', hue='Forecast Type')
        plt.title(f'{station_name} - {model_name.upper()}: Runoff Comparison by Lead Time', fontsize=16)
        plt.xlabel('Lead Time (hours)')
        plt.ylabel('Streamflow (m³/s)')
        plt.legend(title='Forecast Type')
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.savefig(figures_dir / f"{station_name}_{model_name}_runoff_boxplot.png")
        plt.show()

        # --- 2. Metrics Comparison Plots ---
        for metric in ['CC', 'RMSE', 'PBIAS', 'NSE']:
            plt.figure(figsize=(15, 7))
            sns.lineplot(data=metrics_df, x='lead_time', y=metric, hue='type', marker='o', style='type')
            plt.title(f'{station_name} - {model_name.upper()}: {metric} vs. Lead Time', fontsize=16)
            plt.xlabel('Lead Time (hours)')
            plt.ylabel(metric)
            plt.grid(True, which='both', linestyle='--', linewidth=0.5)
            # For NSE, add a zero line for reference
            if metric == 'NSE':
                plt.axhline(0, color='red', linestyle='--', label='NSE = 0 (Baseline)')
            plt.legend()
            plt.tight_layout()
            plt.savefig(figures_dir / f"{station_name}_{model_name}_metric_{metric}.png")
            plt.show()

print("\nAll visualizations are complete and saved.")